# Post-processing

Workflow for processing post-processing experiment result HDF5 dataset and exception metadata.

# Setup and Dependencies

In [ ]:
# Set project root folder, to enable importing project files from subdirectories
from pathlib import Path
import os

path = Path().resolve()
root_path = str(path).split('notebooks')[0]
os.chdir(root_path)

# Force reload of project modules, sometimes necessary for Jupyter kernel
%load_ext autoreload
%autoreload 2

# Display framework versions for easy debugging
%pip show cadCAD
%pip show radcad

In [ ]:
# Import all shared dependencies and setup
from shared import *

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import plotly.io as pio
# pio.renderers.default = "png"
from pprint import pprint

In [ ]:
# Update dataframe display settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

## Experiment Metadata

In [ ]:
experiment_folder = 'experiments/system_model_v3/experiment_monte_carlo'

---

# Post Processing

In [ ]:
def drop_columns(df):
    df.drop(
        ['events', 'uniswap_oracle', 'w_1', 'w_2', 'w_3', 'interest_wiped', 'interest_bitten', 'accrued_interest', 'interest_dripped', 'rai_bitten', 'eth_bitten', 'cdps', 'blockheight', 'sim_metrics', 'optimal_values', 'cdp_metrics'],
        axis=1,
        inplace=True
    )

In [ ]:
with pd.HDFStore(f'{experiment_folder}/experiment_results.hdf5', 'r') as store:
    keys = store.keys()
    print(keys)

In [ ]:
df_raw = pd.read_hdf(f'{experiment_folder}/experiment_results.hdf5', list(filter(lambda x: 'results' in x, keys))[0])

In [ ]:
drop_columns(df_raw)

In [ ]:
df_raw

In [ ]:
from experiments.system_model_v3.post_process import post_process_results
from experiments.system_model_v3.experiment_monte_carlo import SIMULATION_TIMESTEPS, params

Remove substeps, add `set_params` to dataframe, and add post-processing columns:

In [ ]:
df = post_process_results(df_raw, params, set_params=[
    'ki',
    'kp',
    'alpha',
    'liquidation_ratio',
    'controller_enabled',
    'control_period',
    'liquidity_demand_shock',
    'arbitrageur_considers_liquidation_ratio',
    'rescale_target_price'
])

## Save post-processed results with simulation exceptions

In [ ]:
%%capture
processed_results = 'experiments/system_model_v3/experiment_monte_carlo/processed_results.hdf5'

df.to_hdf(
    processed_results,
    key='results'
)

In [ ]:
df_exceptions = pd.read_hdf(f'{experiment_folder}/experiment_results.hdf5', list(filter(lambda x: 'exceptions' in x, keys))[0])
df_exceptions

In [ ]:
%%capture
processed_results = 'experiments/system_model_v3/experiment_monte_carlo/processed_results.hdf5'

df_exceptions.to_hdf(
    processed_results,
    key='exceptions'
)